# 大作业2 网页爬取和解析

## 作业要求：

使用面对对象编程的方式，分别完成以下任务：

1. 从网上抓取一部金庸小说的所有章节，将每个章节对应的网页，保存到本地。（需提交抓取到的网页文件）

2. 在抓取的基础上，对抓取的网页进行处理，提取小说正文，保存到本地。（需提交抓取到的正文文本文件）

3. 对小说正文进行分词，统计词频和人物在每个章节中出现次数（参考作业5和作业6）。并将其分别以csv文件的形式，保存到文件中。（需分别提交两个csv文件）

## 注意事项：

1. 需要使用面向对象的方式完成此次作业。在编程过程中需要定义三个或以上的自定义类，并设计相应的属性、方法。例如，可以针对爬取网页、网页文件解析、分词和词频统计分别设计一个类。

2. 请思考应该如何设计类和类的接口，使得程序的结构和逻辑更加清晰，并且是类中实现的代码有一定的可复用性。例如，使得爬取网页的类能够同时被用来完成抓取所有小说正文和对小说正文进行词频和人物出场次数统计两个不同的任务。除代码（需有一定注释）、需提交的网页/正文/词频统计信息文件之外，还需要提交一个文档，对类和类的接口设计进行简要说明。

3. 类的设计和实现可以参考以下两个样例，并可以通过继承、组合等方式利用样例中实现的类完成作业要求的任务。

4. 进行分词和人物出现次数统计时需要注意添加自定义词典，小说中主要人物列表可参考”金庸小说人物.txt”文件。

5. 短时间内连续访问一个网站可能会触发网站的防护机制，导致网站拒绝相应后续HTML请求。在设计网页爬取代码时应该考虑这一点，避免短时间内发起大量请求。同时，在触发一个网站的保护机制，导致抓取无法进行时，可以自行选择其他站点抓取金庸小说。以下列出了几个可供选择的网站：

https://www.jinyongwang.com/

http://jinyong.zuopinj.com/

http://jinyongxiaoshuo.com/

6. 需要注意中文网页编码问题，在必要时人工指定正确的编码。


## 样例1：获取网页html代码并提取出其中所有超链接

In [1]:
# 安装所需包
!pip install requests
!pip install beautifulsoup4

You are using pip version 19.0.3, however version 21.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 21.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import requests
from bs4 import BeautifulSoup
import time

class BasicAnalyzer(object):
    """
    一个最基本的网页解析器，从response对象中获取text字段
    """
    def parse(self, task_name, r): # 解析response
        return r.text

class Cralwer(object):
    """
    一个用来爬取网页的类，其主要功能是依次抓取URL，并将返回的结果交给后续的解析器（Analyzer）进行处理。
    """
    
    def __init__(self, task_or_tasks, analyzer=BasicAnalyzer(), 
                 headers={}, timeout=30, encoding=None, wait_time=-1): 
        if isinstance(task_or_tasks, str):
            self.tasks = [task_or_tasks]
        if isinstance(task_or_tasks, list) or isinstance(task_or_tasks, tuple):
            self.tasks = list(task_or_tasks)
        print(self.tasks)
        self.analyzer = analyzer
        self.headers = headers
        self.timeout = timeout
        self.encoding = encoding
        self.wait_time = wait_time
        
        # 用于保存抓取请求返回的状态码
        self.response_codes = []
        
        # 用于遍历所有任务的迭代器
        self.__iterator = iter(self.tasks)
    
    def add_tasks(self, task_or_tasks):
        if isinstance(task_or_tasks, str):
            self.tasks.append(task_or_tasks)
        if isinstance(task_or_tasks, list) or isinstance(task_or_tasks, tuple):
            self.tasks += list(task_or_tasks)
    
    def crawl(self):
        """
        该方法会从任务序列中取出下一个抓取任务，调用__process_task方法进行抓取。
        """
        task_uri = next(self.__iterator)
        if self.wait_time > 0:
            print("等待{}秒后开始抓取".format(self.wait_time))
            time.sleep(self.wait_time)
        return self.__process_task(task_uri)
            
    def __process_task(self, task):
        """
        完成task指定的抓取任务。
        """
        if isinstance(task, str): # 如果task是一个字符串，那么task代表要抓取的网页URI
            task_name = None
        elif isinstance(task, tuple) and len(task) == 2: # 如果task是一个长度为2的元组，那么task表示（任务名，网页URI）
            task_name, task = task
        else: # 否则报错
            raise ValueError("无法识别任务:{}".format(task))
        try:
            print(task_name, task)
            r = requests.get(task, headers=self.headers, timeout=self.timeout)
            if self.encoding is not None:
                r.encoding = self.encoding
            self.response_codes.append((task_name, r.status_code))
        except:
            self.response_codes.append((task_name, None)) # 若遇到链接错误等问题，则此次任务的响应状态码为None
            return None
        return self.analyzer.parse(task_name, r) # 将response对象交给analyzer处理，这时会调用__call__方法

    def __iter__(self):
        """
        通过重载__iter__和__next__两个方法，可以使得我们能够通过类似for x in Crawler()这样的方式依次完成所有抓取任务。
        __iter__会在循环遍历开始前被调用，这个方法应该返回一个可遍历对象。
        由于实现了上述两个方法，Crawler对象本身就是一个可遍历对象，所以我们直接返回self。
        """
        return self
    
    def __next__(self):
        """
        __next__方法应该依次返回抓取的结果，因此我们调用crawl()方法，完成队列中的一个抓取任务，返回经过analyzer处理后的结果。
        """
        return self.crawl()
    
    def crawl_all(self):
        """
        完成所有抓取任务，将结果保存到一个list中返回。
        这里我们直接使用列表推导式方法，循环完成抓取。
        注意，这里能将self（即Crawler对象自身）用于列表推导式的原因是
        Crawler对象是一个可遍历对象，即一个实现了__iter__和__next__特殊方法对象。
        """
        return [result for result in self]

        


In [3]:
# 使用上述两个类配合，我们可以抓取一个网页
c = Cralwer([('主页', 'http://jinyongxiaoshuo.com/')], encoding='utf8', wait_time=2)
c.crawl()

[('主页', 'http://jinyongxiaoshuo.com/')]
等待2秒后开始抓取
主页 http://jinyongxiaoshuo.com/


'<!DOCTYPE html>\r\n<html class="no-js" lang="zh-CN">\r\n<head>\r\n<meta charset="UTF-8">\r\n<meta name="viewport" content="width=device-width, initial-scale=1.0">\r\n<meta http-equiv="Cache-Control" content="no-transform" />\r\n<meta http-equiv="Cache-Control" content="no-siteapp" />\r\n<title>金庸小说全集_金庸小说在线阅读第一站，金庸旧版、修订版、新修版小说全集在线阅读</title>\r\n<meta name="keywords" content="金庸小说全集,金庸作品集,金庸小说全文在线阅读,金庸小说三联版,金庸小说修订版,金庸小说新修版" />\r\n<meta name="description" content="金庸小说全集，为您免费提供金庸(旧版、修订版、新修版)小说全集在线阅读。金庸小说全集是内容最齐全完整、最受读者欢迎的金庸小说在线阅读网站。喜欢金庸小说，请认准金庸小说全集网址：JinYongXiaoShuo.COM。" />\r\n<link rel=\'index\' title=\'金庸小说全集\' href=\'http://jinyongxiaoshuo.com\' />\r\n<meta name="robots" content="index,follow"/>\r\n<link rel="profile" href="http://gmpg.org/xfn/11">\r\n<link rel="shortcut icon" href="/css/img/favico.ico" />\r\n<link rel="stylesheet" href="/css/style.css" type="text/css" media="screen" />\r\n<link rel="stylesheet" media="screen and (max-width:600px)" href="/css/mobile.css" type="text/c

In [4]:
# 我们可以通过继承BasicAnalyzer类，为其增添一些功能，例如，我们想获取高瓴人工智能学院官网的学院介绍
class GSAIIntroductionAnalyzer(BasicAnalyzer):
    """
    提取高瓴人工智能学院主页上的学院介绍内容。
    """
    def parse(self, task_name, r): # 解析response
        html_text = super().parse(task_name, r)
        soup = BeautifulSoup(html_text)
        # 如何查找正文需要针对要抓取的网页单独设计
        # 可以通过查看源文件和浏览器”审查元素“等功能查看网页结构
        # 然后按照标签名称、id和class属性、标签在解析树上的位置等方式设计相应的方法查找到所需要的元素。
        parsed_text = "\n".join(
            [p.string for p in soup.find('div', class_="fr").find_all('p') 
                       if p.string is not None]
        )
        return parsed_text
    
c = Cralwer(["http://ai.ruc.edu.cn/overview/intro/index.htm"], 
            analyzer=GSAIIntroductionAnalyzer(),
            headers={'user-agent': 'my-app/0.0.1'}, 
            encoding='utf8',
            wait_time=2
           )
c.crawl_all()


['http://ai.ruc.edu.cn/overview/intro/index.htm']
等待2秒后开始抓取
None http://ai.ruc.edu.cn/overview/intro/index.htm


['“过去未去，未来已来”，在构建人工智能时代的宏大世界观时，在影响人工智能技术发展的历史趋势时，在吸纳和培养人工智能领域的顶尖学者和实践者时，中国人民大学高瓴人工智能学院应运而生，并将扮演至关重要的角色。\n高瓴人工智能学院是中国人民大学下属学院，承担学校人工智能学科的规划与建设，开展本学科和相关交叉学科领域的本、硕、博人才培养和科学研究工作。学院由高瓴资本创始人兼首席执行官、耶鲁大学校董、中国人民大学校友张磊先生捐资支持建设。\n高瓴人工智能学院学术委员会主任由中国工程院原常务副院长、国家新一代人工智能战略咨询委员会主任潘云鹤院士担任，执行院长由中国人民大学信息学院院长文继荣教授担任。\n学院愿景\n打造一所能够影响和塑造未来人工智能时代的世界一流学院，为全球思考并创造“智能而有温度”的未来。\n发展目标\n创新一流体制机制、打造一流师资队伍、培养一流专业人才、产出一流科研成果。\n主要任务\n推动人工智能基础理论和技术研究；探索建立新型交叉研究中心，促进人文社科与人工智能的深度融合；联合各界设立联合研究中心和实验室，与全球知名人工智能企业联合打造专项人才培养计划，鼓励创新和产业化，促进凝聚广泛共识的人工智能全球对话。']

In [5]:
# 我们也可以获取页面上所有的链接，把他们输出到一个csv文件
class LinkAnalyzer(BasicAnalyzer):
    def __init__(self, filename, encodings=None):
        
        self.filename = filename
        
    def parse(self, task_name, r):
        html_text = super().parse(task_name, r)
        soup = BeautifulSoup(html_text, 'html.parser') # 使用自带的解析器，解析上述html文档
        with open(self.filename, 'w') as fout:
            fout.write("锚文本,超链接\n")
            for tag in soup.find_all('a'):
                if tag.string is not None:
                    fout.write("{},{}\n".format(tag.string, tag.get('href', None)))
        
c = Cralwer(["http://jinyong.zuopinj.com/"], 
            analyzer=LinkAnalyzer('links.csv'),
            encoding='utf8',
            wait_time=2
           )
c.crawl_all()

['http://jinyong.zuopinj.com/']
等待2秒后开始抓取
None http://jinyong.zuopinj.com/


[None]

## 样例2：抓取多个网页并保存到本地

In [ ]:
# 我们还可以把所有页面都依次抓取下来并保存到文件里
import os

class FileStorageAnalyzer(BasicAnalyzer):
    def __init__(self, dir_path):
        self.dir_path = dir_path
        self.cnt = 0
    
    def parse(self, task_name, r):
        html_text = super().parse(task_name, r) # 调用父类，获取html text
        
        # 将其保存到文件
        if task_name is not None and isinstance(task_name, str):
            file_path = os.path.join(self.dir_path, "{}.html".format(task_name))
        else:
            file_path = os.path.join(self.dir_path, "{0:04}.html".format(self.cnt))
            self.cnt += 1
        with open(file_path, 'w') as fout:
                fout.write(html_text)
        return html_text

tasks = []
with open("links.csv", 'r') as fin: # 注意这里的links.csv文件是上一个cell运行后生成的文件。
    header = fin.readline()
    for line in fin:
        name, uri = line.strip().split(',')
        if uri.startswith('http://jinyong.zuopinj.com/'):
            tasks.append((name, uri))
        
c = Cralwer(tasks, analyzer=FileStorageAnalyzer("book_pages/"), wait_time=2, encoding='utf8')
c.crawl_all()
print(c.response_codes)

[('金庸作品集', 'http://jinyong.zuopinj.com/'), ('侠客行', 'http://jinyong.zuopinj.com/12/'), ('书剑恩仇录', 'http://jinyong.zuopinj.com/10/'), ('倚天屠龙记', 'http://jinyong.zuopinj.com/7/'), ('笑傲江湖', 'http://jinyong.zuopinj.com/5/'), ('神雕侠侣', 'http://jinyong.zuopinj.com/4/'), ('鹿鼎记', 'http://jinyong.zuopinj.com/3/'), ('天龙八部', 'http://jinyong.zuopinj.com/2/'), ('射雕英雄传', 'http://jinyong.zuopinj.com/1/'), ('鸳鸯刀', 'http://jinyong.zuopinj.com/15/'), ('越女剑', 'http://jinyong.zuopinj.com/14/'), ('白马啸西风', 'http://jinyong.zuopinj.com/13/'), ('侠客行', 'http://jinyong.zuopinj.com/12/'), ('连城诀', 'http://jinyong.zuopinj.com/11/'), ('书剑恩仇录', 'http://jinyong.zuopinj.com/10/'), ('雪山飞狐', 'http://jinyong.zuopinj.com/9/'), ('飞狐外传', 'http://jinyong.zuopinj.com/8/'), ('倚天屠龙记', 'http://jinyong.zuopinj.com/7/'), ('碧血剑', 'http://jinyong.zuopinj.com/6/'), ('笑傲江湖', 'http://jinyong.zuopinj.com/5/'), ('神雕侠侣', 'http://jinyong.zuopinj.com/4/'), ('鹿鼎记', 'http://jinyong.zuopinj.com/3/'), ('天龙八部', 'http://jinyong.zuopinj.com/2/'), ('射雕英